In [1]:
from param_def import chi,sigma,prop_gp,capacities_rate,prop_all_g_prefer,capacities
import numpy as np
from util import *
from itertools import compress
# Import HyperOpt Library
from hyperopt import tpe, hp, fmin
import numpy as np
from BayesEst import bayesian_est

In [2]:
print(capacities_rate)

[0.4, 0.4]


In [3]:
grade_estimated_gr = []
for i in range(len(chi)):
    grade_estimated_gr.append(np.load(f'grade_estimated_gr{i+1}.npy'))
grade_estimated = grades_gr_to_grades_col(grade_estimated_gr)
#%%
stud_pref = []
for i in range(len(chi)):
    stud_pref.append(np.load(f'stud_pref_gr{i+1}.npy').tolist())

In [4]:
def market_clear(Pa, Pb, prop, capA, capB, prefi, prefii, sigmai, sigmaii, cori, corii):
    f1 = prop*prefi*(1 - cdf(Pa, sigmai)) + (1 - prop)*prefii*(1 - cdf(Pa, sigmaii)) + prop*(1 - prefi)*cdfmsi(Pa, Pb, sigmai, cori) + (1 -prop)*(1 - prefii)*cdfmsi(Pa, Pb, sigmaii, corii) - capA
    f2 = prop*(1 - prefi)*(1 - cdf(Pb, sigmai)) + (1 - prop)*(1 - prefii)*(1 - cdf(Pb, sigmaii)) + prop*prefi*cdfmis(Pa, Pb, sigmai, cori) + (1 -prop)*prefii*cdfmis(Pa, Pb, sigmaii, corii) - capB
    return f1, f2

In [5]:
def solv_mc(prop, capA, capB, prefi, prefii, sigmai, sigmaii, cori, corii):
    func = lambda P: market_clear(P[0], P[1], prop, capA, capB, prefi, prefii, sigmai, sigmaii, cori, corii)
    return fsolve(func, x0 = np.array([0.5, 0.5]))

In [6]:
sigma_i = np.sqrt(chi[0]**2 + sigma[0]**2)
sigma_ii = np.sqrt(chi[1]**2 + sigma[1]**2)
cor_i = chi[0]**2/sigma_i**2
cor_ii = chi[1]**2/sigma_ii**2

In [7]:
print(cor_i,cor_ii)

0.8999999999999998 0.4999999999999999


In [8]:
P_A,P_B = solv_mc(prop_gp[0],capacities_rate[0],capacities_rate[1],
                  prop_all_g_prefer[0],prop_all_g_prefer[1],sigma_i,sigma_ii,cor_i,cor_ii)


In [9]:
cutoff_values = [P_A,P_B]

In [10]:
P_A,P_B

(np.float64(-2.394683959950654), np.float64(-1.1792879240188))

In [11]:
welfare_metrics(cutoff_values,grade_estimated,stud_pref)

Proportion of students in group 0 with no offer 0.31
Proportion of students in group 0 with only a second preference offer 0.09
Proportion of students in group 0 with first choice offer 0.60
Proportion of students in group 1 with no offer 0.12
Proportion of students in group 1 with only a second preference offer 0.17
Proportion of students in group 1 with first choice offer 0.71


In [12]:
student_by_col(cutoff_values,grade_estimated,stud_pref)

Number of student admitted to college 1 in group 0: 1941
Number of student admitted to college 2 in group 0: 2165
Number of student admitted to college 1 in group 1: 4064
Number of student admitted to college 2 in group 1: 3768


(6005, 5933)

In [13]:
print(capacities)

[5937, 5937]


In [14]:
new_cutoff_values = [-2.35113263,-1.80450201]
new_cutoff_values

[-2.35113263, -1.80450201]

In [15]:
updated_grade_estimated = bayes_update_grade(new_cutoff_values[0],new_cutoff_values[1],grade_estimated,chi,sigma,bayes_type='right')

In [16]:
welfare_metrics(new_cutoff_values,updated_grade_estimated,stud_pref)

Proportion of students in group 0 with no offer 0.33
Proportion of students in group 0 with only a second preference offer 0.11
Proportion of students in group 0 with first choice offer 0.56
Proportion of students in group 1 with no offer 0.11
Proportion of students in group 1 with only a second preference offer 0.15
Proportion of students in group 1 with first choice offer 0.74


In [17]:
student_by_col(new_cutoff_values,updated_grade_estimated,stud_pref)

Number of student admitted to college 1 in group 0: 2129
Number of student admitted to college 2 in group 0: 1837
Number of student admitted to college 1 in group 1: 3836
Number of student admitted to college 2 in group 1: 4082


(5965, 5919)